# Clash of Clans Automation

This is a work in progress to automate the clash of clans game.
You'll need:
* Windows
* Bluestacks app: https://www.bluestacks.com/download.html

## Grab screen

Got the script from pygta5 sentdex: https://github.com/Sentdex/pygta5/blob/master/grabscreen.py

In [1]:
# Done by Frannecklp

import cv2
import numpy as np
import win32gui
import win32ui
import win32con
import win32api

In [2]:
def grab_screen(region=None):
    hwin = win32gui.GetDesktopWindow()

    if region:
        left, top, x2, y2 = region
        width = x2 - left + 1
        height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)

    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height, width, 4)

    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

In [3]:
def show_image(image):
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [33]:
def grab_coc():
    image = grab_screen()
    # Use MPos to get the coordinates: https://sourceforge.net/projects/mpos/
    # Top-left
    # 265, 106
    # Bottom-right
    # 1718, 938
    # Add 1920 to width slices because of dual monitors
    # Crop width and height
    width = slice(2185, 3638)
    height = slice(106, 938)

    image = image[height, width]
    return image

In [24]:
show_image(image)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [9]:
# while True:
#     image = grab_screen()

#     cv2.imshow("Image", image)
#     cv2.waitKey(15)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


KeyboardInterrupt: 

## Move Mouse

In [19]:
import pyautogui
import time

In [9]:
pyautogui.moveTo(100, 150)

In [18]:
clash_of_clans_loc = (1100, 280)
okay_loc = (1050, 650)

In [11]:
pyautogui.moveTo(clash_of_clans_loc)

In [23]:
pyautogui.click(clash_of_clans_loc)

In [22]:
pyautogui.click(clash_of_clans_loc)

pyautogui.hotkey('ctrl','shift','2')
time.sleep(0.5)
pyautogui.click(okay_loc)

## Template matching

https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html

In [25]:
import cv2

In [26]:
import numpy as np

In [34]:
img_rgb = grab_coc()
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('images/elixir_collector.jpg', 0)
w, h = template.shape[::-1]

result = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
# print(result)
threshold = 0.8
loc = np.where(result >= threshold)
# print(loc)
for pt in zip(*loc[::-1]):
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
cv2.imwrite('images/res.png',img_rgb)

C:\Users\patta\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


True